# Sistemas de recomendación: filtrado basado en contenido

Filtrado basado en contenido utilizando red neuronal (Deep Learning) para crear un sistema de recomendación de películas.


## Esquema
- [ 1 - Paquetes ](#1)
- [ 2 - Conjunto de datos de clasificaciones de películas ](#2)
- [ 3 - Filtrado basado en contenido con una red neuronal](#3)
- [ 3.1 Datos de entrenamiento](#3.1)
- [ 3.2 Preparación de los datos de entrenamiento](#3.2)
- [ 4 - Red neuronal para filtrado basado en contenido](#4)
- [ Implementación Algoritmo 1](#ex01)
- [ 5 - Predicciones](#5)
- [ 5.1 - Predicciones para un nuevo usuario](#5.1)
- [ 5.2 - Predicciones para un usuario existente.](#5.2)
- [ 5.3 - Búsqueda de elementos similares](#5.3)
- [ Implementación Algoritmo 2](#ex02)
- [ 6 - Conclusiones ](#6)


## 1. Packages 

In [12]:
# Librerias, Visualizacion, Procesamiento de datos

import pandas as pd
import numpy as np
import numpy.ma as ma

# Tensorflow, Red Neuronal
import tensorflow as tf
from tensorflow import keras

# Procesameinto de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

# Libreria Personalizada:  # Leer y Procesar datos
from recsysNN_utils import *

pd.set_option("display.precision", 1)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')


## 2 Conjunto de datos de calificaciones de películas
El conjunto de datos se deriva del conjunto de datos [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/).

[F. Maxwell Harper y Joseph A. Konstan. 2015. Los conjuntos de datos de MovieLens: historia y contexto. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]

El conjunto de datos original tiene aproximadamente 9000 películas calificadas por 600 usuarios con calificaciones en una escala de 0,5 a 5 en incrementos de 0,5 pasos. El conjunto de datos se ha reducido en tamaño para centrarse en películas de los años posteriores al 2000 y en géneros populares. El conjunto de datos reducido tiene $n_u = 397$ usuarios, $n_m = 847$ películas y 25521 calificaciones. Para cada película, el conjunto de datos proporciona un título de película, fecha de lanzamiento y uno o más géneros. Por ejemplo, "Toy Story 3" se estrenó en 2010 y tiene varios géneros: "Aventura|Animación|Infantil|Comedia|Fantasía". Este conjunto de datos contiene poca información sobre los usuarios aparte de sus calificaciones. Este conjunto de datos se utiliza para crear vectores de entrenamiento para las redes neuronales que se describen a continuación.

In [2]:
# Set de datos

top10_df = pd.read_csv("./data/content_top10_df.csv")
bygenre_df = pd.read_csv("./data/content_bygenre_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.1,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.0,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.1,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.9,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.2,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.8,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.1,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,4.0,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.9,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.2,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


La siguiente tabla muestra información ordenada por género. La cantidad de calificaciones por género varía considerablemente. Tenga en cuenta que una película puede tener varios géneros, por lo que la suma de las calificaciones que se muestran a continuación es mayor que la cantidad de calificaciones originales.

In [3]:
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.4,10377
1,Adventure,234,3.4,8785
2,Animation,76,3.6,2588
3,Children,69,3.4,2472
4,Comedy,326,3.4,8911
5,Crime,139,3.5,4671
6,Documentary,13,3.8,280
7,Drama,342,3.6,10201
8,Fantasy,124,3.4,4468
9,Horror,56,3.2,1345


## 3. Filtrado basado en contenido con una red neuronal

filtrado colaborativo = 2 vectoresdos vectores, un vector de usuario y un vector de elemento/película cuyo producto escalar predeciría una calificación. Los vectores se derivaron únicamente de las calificaciones.

El filtrado basado en contenido también genera un vector de características de usuario y película, pero reconoce que puede haber otra información disponible sobre el usuario o la película que pueda mejorar la predicción. La información adicional se proporciona a una red neuronal que luego genera el vector de usuario y película como se muestra a continuación.

### 3.1 Datos de entrenamiento
El contenido de la película proporcionado a la red es una combinación de los datos originales y algunas "características diseñadas". Características originales son el año en que se estrenó la película y el género de la película se presenta como un vector único. Hay 14 géneros. La característica diseñada es una calificación promedio derivada de las calificaciones de los usuarios.

El contenido del usuario está compuesto de características diseñadas. Se calcula una calificación promedio por género por usuario. Además, están disponibles una identificación de usuario, un recuento de calificaciones y un promedio de calificaciones, pero no se incluyen en el contenido de entrenamiento o predicción. Se incluyen con el conjunto de datos porque son útiles para interpretar los datos.

El conjunto de entrenamiento consta de todas las calificaciones realizadas por los usuarios en el conjunto de datos. Algunas calificaciones se repiten para aumentar la cantidad de ejemplos de entrenamiento de géneros subrepresentados. El conjunto de entrenamiento se divide en dos matrices con la misma cantidad de entradas, una matriz de usuario y una matriz de película/artículo.

In [4]:
# Cargar datos, establecer variables de configuración
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3 # eliminar ID de usuario, recuento de calificaciones y calificación promedio para el entrenamiento
num_item_features = item_train.shape[1] - 1  # eliminar el ID de la película a la hora de Entrenar


uvs = 3 # inicio del vector de género del usuario
ivs = 3 # inicio del vector de género del elemento
u_s = 3 # inicio de las columnas a utilizar en el entrenamiento, usuario
i_s = 1 # inicio de las columnas a utilizar en el entrenamiento, elementos
print(f"Número de vectores de entrenamiento: {len(item_train)}")

Number of training vectors: 50884


In [5]:
# primeras entradas en la matriz de entrenamiento del usuario.
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


Algunas de las características de usuario y de elemento/película no se utilizan en el entrenamiento. En la tabla anterior, las características entre corchetes "[]", como "identificación de usuario", "número de calificaciones" y "promedio de calificaciones", no se incluyen cuando se entrena y utiliza el modelo.
Arriba puede ver el promedio de calificaciones por género para el usuario 2. Las entradas cero son géneros que el usuario no ha calificado. El vector de usuario es el mismo para todas las películas calificadas por un usuario.
Veamos las primeras entradas de la matriz de películas/elementos.

In [6]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


Arriba, la matriz de películas contiene el año en que se estrenó la película, la calificación promedio y un indicador para cada género potencial. El indicador es uno para cada género que se aplica a la película. El identificador de la película no se utiliza en el entrenamiento, pero es útil para interpretar los datos.

In [7]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


El objetivo, y, es la calificación de la película dada por el usuario. 

Arriba, podemos ver que la película 6874 es una película de acción, crimen y suspenso estrenada en 2003. El usuario 2 califica las películas de acción con un promedio de 3,9. Los usuarios de MovieLens le dieron a la película una calificación promedio de 4. 'y' es 4, lo que indica que el usuario 2 también calificó la película 6874 con un 4. Un solo ejemplo de entrenamiento consta de una fila de las matrices de usuario y elemento y una calificación de y_train.

### 3.2 Preparación de los datos de entrenamiento

- Escalar características de entrada (X). 

- Escalar calificaciones objetivo (y). Nota: Min Max Scaler escalar objetivo para que esté entre -1 y 1.

In [8]:
# Datos de Entrenamiento
item_train_unscaled = item_train
user_train_unscaled = user_train
# Datos Objetivos
y_train_unscaled    = y_train

# Para train
scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

# Para Usuarios
scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

# Para objetivo (calificaciones)
scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


### Dividir datos en conjuntos de entrenamiento y prueba.

In [9]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


Nota: Los datos escalados y mezclados ahora tienen una media de cero.

In [10]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


## 4. Red neuronal para filtrado basado en contenido
La red Neuronal, Tendrá 2 redes que se combinan mediante un producto escalar (Para este Poryecto serán idénticas).

Nota:** Tener en cuenta que estas redes no necesitan ser iguales. Si el contenido del usuario fuera sustancialmente más grande que el contenido de la película, se podría optar por aumentar la complejidad de la red del usuario en relación con la red de la película. En este caso, el contenido es similar, por lo que las redes son iguales.**



In [13]:
num_outputs = 32
tf.random.set_seed(1)

user_NN = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense(256, activation= 'relu'),
    tf.keras.layers.Dense(128, activation= 'relu'),
    tf.keras.layers.Dense(num_outputs),
])

item_NN = tf.keras.models.Sequential([
  
    
     tf.keras.layers.Dense(256, activation= 'relu'),
     tf.keras.layers.Dense(128, activation= 'relu'),
     tf.keras.layers.Dense(num_outputs),

])

# crea la entrada del usuario y apunta a la red base
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# crea la entrada del elemento y apunta a la red base
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# Calcular el producto escalar de los dos vectores vu y vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# especifica las entradas y salidas del modelo
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 32)           40864       input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           41376       input_2[0][0]                    
______________________________________________________________________________________________

In [15]:
tf.random.set_seed(1)

# Definir la perdida y el Optimizador
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [16]:
tf.random.set_seed(1)
# El modelo recive 2 conjunto de X_Etrenamiento 
# para usuarios e item, y solo uno de y_entrenamiento usando la API Secuencial
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Train on 40707 samples
Epoch 1/30
40707/40707 [==============================] - 5s 123us/sample - loss: 0.1232
Epoch 2/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.1146
Epoch 3/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.1089
Epoch 4/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.1039
Epoch 5/30
40707/40707 [==============================] - 5s 112us/sample - loss: 0.1001
Epoch 6/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.0973
Epoch 7/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.0956
Epoch 8/30
40707/40707 [==============================] - 5s 112us/sample - loss: 0.0935
Epoch 9/30
40707/40707 [==============================] - 5s 111us/sample - loss: 0.0916
Epoch 10/30
40707/40707 [==============================] - 5s 112us/sample - loss: 0.0897
Epoch 11/30
40707/40707 [==============================] - 5s 113us/sample - loss: 0.0

In [17]:
# Evaluar modelo, Visualizar pérdida en los datos de prueba.

model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

10177/10177 [==============================] - 0s 37us/sample - loss: 0.0815


0.08146006993124337

Nota: Es comparable a la pérdida de entrenamiento, lo que indica que el modelo no se ha ajustado sustancialmente a los datos de entrenamiento.

## 5. Predicciones
realizar predicciones en diversas circunstancias.

### 5.1 Predicciones para un nuevo usuario:
- Hacer Lo siguiente:

1. Crearemos nuevo usuario (el modelo sugerira películas para ese usuario Nuevo). 
2. Probado esto en el contenido del usuario de ejemplo, (cambiar contenido con propias preferencias y vea lo que sugiere el modelo). 

Nota: Importante Tener en cuenta que las calificaciones están entre 0,5 y 5,0, inclusive, en incrementos de medio paso.

In [18]:
# Nuevo Usuario con carcatetisticas Porpias de ese Usuario

new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

- Para este Nuevo Usuario:

- Los géneros que el nuevo usuario le gustan son aventura y fantasía.

- El modelo sugerirá películas que también le gustan aventura y fantasía (películas mejor valoradas por el nuevo usuario.).

- Las películas que el nuevo usuario le gustan son:

- The Shawshank Redemption (1994)

- The Godfather (1972)

- Pulp Fiction (1994)

- The Dark Knight (2008)

- La pérdida es alta, pero la cantidad de calificaciones es baja, lo que indica que el nuevo usuario podría estar encantado con estas películas.

- Nota: Este modelo puede sugerir otras películas, pero no todas podrían gustarle. Para obtener recomendaciones más personalizadas, se podría incorporar más información sobre el nuevo usuario, como su historial de calificaciones, géneros preferidos, etc.


**MODELO: A continuación, utilizara un conjunto de vectores de películas/elementos, `item_vecs`, que tienen un vector para cada película del conjunto de entrenamiento/prueba. Esto se combina con el nuevo vector de usuario anterior y los vectores escalados se utilizan para predecir las calificaciones de todas las películas.**

In [19]:
# generar y replicar el vector de usuario para que coincida con el número de películas en el conjunto de datos.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# escalamos nuestros vectores de usuario y de elemento
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# hacer una predicción
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# predicción y sin escala
y_pu = scalerTarget.inverse_transform(y_p)

# ordenar los resultados, la predicción más alta primero
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  # negate para obtener la calificación más alta primero
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  # Usando vectores sin escala para visualización

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

y_p,movie id,rating ave,title,genres
4.5,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.4,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.3,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.3,81834,4,Harry Potter and the Deathly Hallows: Part 1 (2010),Action|Adventure|Fantasy
4.3,59387,4,"Fall, The (2006)",Adventure|Drama|Fantasy
4.3,5952,4,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
4.3,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.3,54259,3.6,Stardust (2007),Adventure|Comedy|Fantasy|Romance


<a name="5.2"></a>
### 5.2 - Predicciones para un usuario existente.
las predicciones para el "usuario 2", (uno de los usuarios del conjunto de datos). Podemos comparar las calificaciones predichas con las calificaciones del modelo.

In [20]:
uid = 2 
# formar un conjunto de vectores de usuario. Este es el mismo vector, transformado y repetido.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# escalamos nuestros vectores de usuario y de elemento
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# hacer una predicción
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# predicción y sin escala
y_pu = scalerTarget.inverse_transform(y_p)

# ordenar los resultados, la predicción más alta primero
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  # negativo para obtener la calificación más alta primero
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #Usando vectores sin escala para visualización
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

#imprime predicciones ordenadas para películas calificadas por el usuario
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.5,5.0,2,[4.0],4.3,80906,Inside Job (2010),Documentary
4.2,3.5,2,"[4.0,4.0]",3.9,99114,Django Unchained (2012),Action|Drama
4.1,4.5,2,"[4.0,4.0]",4.1,68157,Inglourious Basterds (2009),Action|Drama
4.1,3.5,2,"[4.0,3.9,3.9]",3.9,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller
4.0,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
4.0,4.0,2,"[4.1,4.0,3.9]",4.3,48516,"Departed, The (2006)",Crime|Drama|Thriller
4.0,4.5,2,"[4.0,4.1,4.0]",4.2,58559,"Dark Knight, The (2008)",Action|Crime|Drama
4.0,4.0,2,"[4.0,4.1,3.9]",4.0,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
4.0,3.5,2,"[4.0,4.1,4.0,3.9]",3.8,8798,Collateral (2004),Action|Crime|Drama|Thriller
3.9,5.0,2,"[4.0,4.1,4.0]",3.9,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama


La predicción del modelo generalmente se encuentra dentro de un margen de error de la calificación real, aunque no es un predictor muy preciso de cómo un usuario califica películas específicas. Esto es especialmente cierto si la calificación del usuario es significativamente diferente del promedio del género del usuario. Puede variar la identificación de usuario anterior para probar diferentes usuarios. No se utilizaron todas las identificaciones de usuario en el conjunto de entrenamiento.

<a name="5.3"></a>
### 5.3 - Búsqueda de elementos similares
La red neuronal anterior produce dos vectores de características, un vector de características de usuario $v_u$ y un vector de características de película, $v_m$. Se trata de 32 vectores de entrada cuyos valores son difíciles de interpretar. Sin embargo, los elementos similares tendrán vectores similares. Esta información se puede utilizar para hacer recomendaciones. Por ejemplo, si un usuario ha calificado muy bien "Toy Story 3", se podrían recomendar películas similares seleccionando películas con vectores de características de película similares.

Una medida de similitud es la distancia al cuadrado entre los dos vectores $ \mathbf{v_m^{(k)}}$ y $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

<a name="ex02"></a>
### Función para calcular la distancia al cuadrado.

In [46]:

def sq_dist(a,b):
    """
    Devuelve la distancia al cuadrado entre dos vectores
    Argumentos:
    a (ndarray (n,)): vector con n características
    b (ndarray (n,)): vector con n características
    Devuelve:
    d (float) : distancia
    """
    
    #con Numpy
    #d = np.sum((a - b)**2)
    #d= np.sum(np.square(a - b))
    
    # con tensorflow
    d= tf.reduce_sum(tf.square(a - b))
     
    return d

In [47]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

#squared distance between a1 and b1: 0.000    
#squared distance between a2 and b2: 0.030   
#squared distance between a3 and b3: 2.000

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


Se puede calcular una matriz de distancias entre películas una vez que se entrena el modelo y luego reutilizarla para nuevas recomendaciones sin volver a entrenarlo. El primer paso, una vez que se entrena un modelo, es obtener el vector de características de la película, $v_m$, para cada una de las películas. Para ello, utilizaremos el `item_NN` entrenado y crearemos un modelo pequeño que nos permita ejecutar los vectores de la película a través de él para generar $v_m$.

In [49]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use e elemento entrenado item_NN, Usar
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # Incorporar normalización como se hizo en el modelo original.
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           41376       input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Squa [(None, 32)]         0           sequential_1[1][0]               
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Sum  [(None, 1)]          0           tf_op_layer_l2_normalize_2/Square
____________________________________________________________________________________________

Una vez que tenga un modelo de película, puede crear un conjunto de vectores de características de película utilizando el modelo para predecir utilizando un conjunto de vectores de elemento/película como entrada. `item_vecs` es un conjunto de todos los vectores de película. Debe escalarse para usarse con el modelo entrenado. El resultado de la predicción es un vector de características de 32 entradas para cada película.

In [50]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

size of all predicted movie feature vectors: (847, 32)


Calculemos ahora una matriz de la distancia al cuadrado entre cada vector de características de la película y todos los demás vectores de características de la película:
<figure>
    <left> <img src="./images/distmatrix.PNG"   style="width:400px;height:225px;" ></center>
</figure>

Luego podemos encontrar la película más cercana buscando el mínimo a lo largo de cada fila. Usaremos [matrices enmascaradas de numpy](https://numpy.org/doc/1.21/user/tutorial-ma.html) para evitar seleccionar la misma película. Los valores enmascarados a lo largo de la diagonal no se incluirán en el cálculo.

In [51]:
count = 50 # número de películas a mostrar
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # enmascarar la diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

movie1,genres,movie2,genres
Save the Last Dance (2001),Drama|Romance,Mona Lisa Smile (2003),Drama|Romance
"Wedding Planner, The (2001)",Comedy|Romance,Mr. Deeds (2002),Comedy|Romance
Hannibal (2001),Horror|Thriller,Final Destination 2 (2003),Horror|Thriller
Saving Silverman (Evil Woman) (2001),Comedy|Romance,Down with Love (2003),Comedy|Romance
Down to Earth (2001),Comedy|Fantasy|Romance,Bewitched (2005),Comedy|Fantasy|Romance
"Mexican, The (2001)",Action|Comedy,Rush Hour 2 (2001),Action|Comedy
15 Minutes (2001),Thriller,Panic Room (2002),Thriller
Enemy at the Gates (2001),Drama,Kung Fu Hustle (Gong fu) (2004),Drama
Heartbreakers (2001),Comedy|Crime|Romance,Fun with Dick and Jane (2005),Comedy|Crime|Romance
Spy Kids (2001),Action|Adventure|Children|Comedy,"Tuxedo, The (2002)",Action|Adventure|Children|Comedy


Los resultados muestran que el modelo generalmente sugerirá una película con géneros similares.

6. Sitema De Recomendacion: Estos Sistemas pueden ser divididos en dos tipos principales:

1. Recomendación basada en contenido:
Un sistema de recomendación basado en contenido es un tipo de sistema de recomendación que utiliza el contenido del artículo (o objeto) para predecir su popularidad o relevancia. Este tipo de sistema puede ser utilizado para recomendar artículos similares al que un usuario está interesado, o para recomendar artículos que podrían gustarle al usuario según sus preferencias y intereses.

Ejemplos de sistemas de recomendación basados en contenido: Netflix, Amazon, Spotify, Hulu, etc.

2. Recomendación basada en usuarios:

Un sistema de recomendación basado en usuarios es un tipo de sistema de recomendación que utiliza la información del comportamiento de los usuarios para predecir su preferencia o interés en un nuevo artículo. Este tipo de sistema puede ser utilizado para recomendar artículos que podrían gustarle al usuario según sus preferencias y intereses.

Ejemplos de sistemas de recomendación basados en usuarios: Amazon, Facebook, Google My Business, Netflix, etc.

3. Recomendación basada en similitud:

Este tipo de sistema utiliza una medida de similitud entre los artículos para recomendar aquellos que podrían gustarle más al usuario. La medida de similitud puede ser el producto escalar de los vectores de características de los artículos, la distancia euclidiana, la correlación de Pearson, la correlación de Spearman, la distancia coseno, etc.

Ejemplos de sistemas de recomendación basados en similitud: Amazon, Netflix, Spotify, Hulu, etc.

4. Recomendación basada en contenido y usuarios:

Este tipo de sistema utiliza la información del comportamiento de los usuarios y el contenido del artículo para predecir su popularidad o relevancia. Este tipo de sistema puede ser utilizado para recomendar artículos que podrían gustarle al usuario según sus preferencias y intereses.

Ejemplos de sistemas de recomendación basados en contenido y usuarios: Amazon, Netflix, Spotify, Hulu, etc.


Un ejemplo de cómo se puede implementar un sistema de recomendación basado en contenido basada en similitud es utilizando la distancia euclidiana:

**Nota:Esta estructura es la base de muchos sistemas de recomendación comerciales. El contenido del usuario se puede ampliar en gran medida para incorporar más información sobre el usuario si está disponible. Los artículos no se limitan a películas. Esto se puede utilizar para recomendar cualquier artículo, libros, automóviles o artículos que sean similares a un artículo en su "carrito de compras".**

### Para implementar un sistema de recomendación, es importante tener en cuenta las siguientes consideraciones:

- Se debe seleccionar un conjunto de artículos de referencia para calcular la similitud.

- Se debe seleccionar un método de similitud adecuado para calcular la similitud entre los artículos. La distancia euclidiana es común, pero también se puede utilizar la distancia de Manhattan, la distancia de Minkowski, la distancia de Chebyshev, etc.

- Se debe seleccionar un algoritmo de agrupamiento para agrupar los artículos similares. Los algoritmos más comunes incluyen K-means, Mean-Shift, DBSCAN, etc.

- Se debe seleccionar un algoritmo de recomendación para predecir la popularidad o relevancia de un nuevo artículo. Los algoritmos más comunes incluyen la regresión lineal, la regresión logística, el modelo de vecinos más cercanos, etc.

- Se debe seleccionar un criterio de selección de artículos recomendados para un usuario. Por ejemplo, se puede seleccionar los artículos más populares, los artículos más recientes, los artículos más similares al usuario, etc.

- Se debe seleccionar un criterio de calificación para evaluar el rendimiento del sistema de recomendación. Por ejemplo, se puede utilizar la precisión, la exhaustividad, el recall, el F1 score, etc.

- Se debe seleccionar un conjunto de datos de entrenamiento y un conjunto de datos de prueba para evaluar el rendimiento del sistema de recomendación.

- Se debe seleccionar un sistema de almacenamiento para almacenar y recuperar la información de los usuarios y los artículos. Los sistemas más comunes incluyen las bases de datos relacionales, las bases de datos NoSQL, las bases de datos de tiempo real, etc.